# **About Notebook**


This notebook contains the testing of 3 unique image classification models inlcuding ResNet50, EfficentNet50 , and InceptionV3. 

Each model will be tested and evaluated using XAI and standard evaluation tecniques. Attempts will be made to make this as reproduceable as possible, in adittion to this, models will use as similar Tuning as possible to make the comparison fair and consistent; but changes will be made where necssary to ensure model tranining can persist.

A list of the models used and their performance accuracy will be below. 

## **Models Overview**

* ResNet50 shows a balanced yet low performance with an overall accuracy of 64.26%. It struggles with correctly identifying 'Normal' cases but performs well with 'Pneumonia' cases.

* EfficientNetB0 has an overall accuracy of 62.50%. It completely fails to identify 'Normal' cases while perfectly identifying 'Pneumonia' cases.

* InceptionV3 is the best-performing model with a validation accuracy of 77.08%. It shows a better balance between identifying 'Normal' and 'Pneumonia' cases compared to the other models, though it still has room for improvement in reducing false negatives for 'Pneumonia'.

### **Models Can be Found Here** 
[Keras Applications](https://keras.io/api/applications/)

[Pytorch Image Models](https://paperswithcode.com/lib/timm)

---

## **Import Packages & Libraries**

In [ ]:
# Importing necessary packages and libraries

# Operating system interface for file and directory management
import os 

# Base python library for randomisation of data
import random

# Numerical and data manipulation libraries
import numpy as np 
import pandas as pd 
import cv2  # OpenCV for image processing

# Visualisation Package for Data insights
import matplotlib.pyplot as plt

# TensorFlow Keras modules for deep learning model building and preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow import data as tf_data
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50, EfficientNetB0, InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.imagenet_utils import decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image as keras_image
import matplotlib.patches as patches

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Packages for visualising model Evaluations
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from skimage.segmentation import mark_boundaries
from skimage import io
from lime import lime_image

## **Data Directories**

Relevent Links for coding insperation

[tf.keras.preprocessing.image_dataset_from_directory](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory)

[flow_from_directory](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

In [ ]:
# Paths to sets within the directories
train_path = '/kaggle/input/chest_xray/train'
test_path = '/kaggle/input/chest_xray/test'
val_path = '/kaggle/input/chest_xray/val'

In [ ]:
# Define the data augmentation model
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
])

# Optimize data loading
def load_data(directory, img_size=(150, 125), batch_size=32, augment=False):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        directory,
        labels='inferred',
        label_mode='binary',
        color_mode='rgb',
        image_size=img_size,
        batch_size=batch_size,
        crop_to_aspect_ratio=True, 
        seed=42
    )

    if augment:
        # Augment the train dataset to prevent overfitting
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y))
    
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

## **Make Training Sets**

In [ ]:
train_ds = load_data(train_path, augment=True)
test_ds = load_data(test_path)
val_ds = load_data(val_path)

## **Train Model Functions for Simplicity**

[Call Backs from keras](https://keras.io/guides/writing_your_own_callbacks/)

In [ ]:
# Stop training early if 'val_loss' doesn't improve for 6 epochs
# Also, restore the best model weights from the epoch with the lowest 'val_loss'
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

# Reduce learning rate when 'val_loss' stops improving
# Reduce the learning rate by a factor of 0.2 after 3 epochs of no improvement
reduced_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001) 

callbacks = [early_stopping, reduced_plateau]

In [ ]:
def train_top_layers(model, train_generator, val_generator):
    """
    model that can be used to simplify training models based on keras typical transfering workflow tranining
    
    Args: 
        Model: the model that you wish to use to train
        train_generator: you x_train data either exclusively x_train data or a training dataset
        val_generator: the validation data set to compare with the train set
        
    Returns: 
        history: for model evaluation 
    
    """
    
    model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])
    
    history = model.fit(
        train_generator,
        epochs=2, 
        validation_data=val_generator, 
        callbacks=callbacks
    )
    
    return history

In [ ]:
def fine_tune(model, train_generator, val_generator): 
    """
    used to further imporove a trained mode
    Args: 
        model: the tranining model
        train_generator: the tranining dataset
        val_data: the set to compare agains the train set
        
    Returns: 
        history: for model evaluation 
    
    """
    model.trainable = True

    model.compile(
        optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
        loss=keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[keras.metrics.BinaryAccuracy()],
    )


    history = model.fit(
        train_generator, 
        epochs=12, 
        validation_data=val_generator, 
        callbacks=callbacks)
    
    return history

# **Models**

These models are all trained based on the keras typical transfer-learning workflow exmaple to keep the test fair 

Typical transfer-learning workflow: [The typical transfer-learning workflow](https://keras.io/guides/transfer_learning/)

## **Basic Model Building Function**

In [ ]:
def build_model(base_model, img_size=(150, 125, None), num_classes=1):
    """
    Simple function based on keras typical transfering workflow tranining to make it easier to implement existing models
    
    Args: 
        base_model: the model that  you wish to build the structure for
        img_size: ideally this would be done in your preprocessing but you can use this to format the image sizes
        num_classes: the classes used for the model architechture
        
    Returns: 
        model: the built model ready for tranining
    """
    base_model.trainable = False  # Freeze the base model
    inputs = keras.Input(shape=img_size)
    scale_layer = keras.layers.Rescaling(scale=1/127.5, offset=-1)  # Normalize input
    x = scale_layer(inputs)
    x = base_model(x, training=False)  # Ensure base model runs in inference mode
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
    outputs = keras.layers.Dense(num_classes, activation='sigmoid' if num_classes == 1 else 'softmax')(x)  # Adjust for binary or multi-class classification
    model = keras.Model(inputs, outputs)
    return model

## **Evaluation Functions**

Here are some functions to simplify to reproduce easy evaluation techniques for the models.

In [ ]:
def generate_confusion_matrix(model, test_generator): 
    """
    generates a confusion matrix and displays the matrix plot
    Args: 
        model: the model that is being tested
        test_generator: the test dataset
        
    """
    
    # Generate predictions
    true_labels  = np.concatenate([y for x, y in test_generator], axis=0)
    predicted_labels  = np.concatenate([model.predict(x) for x, y in test_generator], axis=0)
    predicted_labels  = np.round(predicted_labels).astype(int)

    # Compute confusion matrix
    confusion_mtx  = confusion_matrix(true_labels, predicted_labels)
    display  = ConfusionMatrixDisplay(confusion_matrix=confusion_mtx, display_labels=['Normal', 'Pneumonia'])

    # Display confusion matrix
    display.plot(cmap='Blues')
    plt.show()

## **ResNet50**

The ResNet50 employs residual blocks that use shortcut connections to jump over some layers. This design helps mitigate the vanishing gradient problem, making it easier to train very deep networks. ResNet50 uses 50 layers, hence the name ResNet50, the user of more layers leads to deeper networks, designed to extract hierarchical features through the layers.

Read more about the ResNet Architechture here: [ResNet and ResNetV2](https://keras.io/api/applications/resnet/)

### **Build Model**

In [ ]:
# The base model for ResNet50 
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 125, 3))
    
# The built model from the function build_model
resnet50_model = build_model(base_model)

### **Train Model**

In [ ]:
# First itteration of tranining for the ResNet50 model
resnet50_model_history = train_top_layers(resnet50_model, train_ds, val_ds)

### **Fine Tune ResNet50 Model**

In [ ]:
# Fine Tuning the model with the fine_tune function to enhance its performance
resnet50_model_history = fine_tune(resnet50_model, train_ds, val_ds)

The ResNet50 model demonstrated strong training performance with nearly 100% accuracy on the training set. However, the evaluation has revealed a lower test accuracy of 81.56%, highlighting issues with overfitting and misclassification. The confusion matrix further elucidates the model's tendency to misclassify normal cases as pneumonia, indicating a need for improved specificity. Further fine-tuning an validation would enhance the model's robustness and reduce misclassification.

---

## **EfficientNetB0 Model**

Unlike ResNet50 EfficientNetB0 uses compound scaling that uniformly scales the network's width, depth and resolution using a set of fixed coefficients. This method allows, EfficientNet to achieve better performance with fewer parameters. It is also designed to be computationally efficient, striking a balance between performance and resource usage. This makes it suitable for environments with limited computational resources.

Read About Model Scaling for Convolution Neural Networks here: [EfficientNet](https://arxiv.org/pdf/1905.11946)

### **Build Model**

In [ ]:
# Base model for the EfficientNetB0 build
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(150, 125, 3))

# Built EfficientNetb0 model
efficientNetB0_model = build_model(base_model)

### **Train Model**

In [ ]:
# Itteration one of traning the EfficientNet model
efficientNetB0_model_history = train_top_layers(efficientNetB0_model, train_ds, val_ds)

### **Fine Tune EfficientNetB0 Model**

In [ ]:
# Fine tuning the moderl for improved performance
efficientNetB0_model_history = fine_tune(efficientNetB0_model, train_ds, val_ds)

## **Inceptionv3 Model**

InceptionV3 uses a network-in-network out approach with mixed convolutions in the same layer, allowing it to capture multi-scale features effectively. It was designed to be computationally efficient while maintaining high accuracy, similar to EfficentNetB0 but using a different architectural approach. The combination of mixed convolutions, factorized convolutions, an auxiliary classifier for enhanced performance and training efficiency.

Read more about InceptionV3 here: [Inception V3 Model Architecture
](https://iq.opengenus.org/inception-v3-model-architecture/)

### **Build Model**

In [ ]:
# Base model for the InceptionV3 build
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 125, 3))

# Built EfficientNetb0 model
inceptionv3_model = build_model(base_model)

### **Train Model**

In [ ]:
# Itteration one of traning the InceptionV3 model
inceptionv3_model_history = train_top_layers(inceptionv3_model, train_ds, val_ds)

### **Fine Tune Model**

In [ ]:
# Fine tuning the InceptionV3 model for improved performance
inceptionv3_model_history = fine_tune(inceptionv3_model, train_ds, val_ds)

## **Evaluation of Models**

Here we compare the architectures and performances of three models: ResNet50, EfficientNetB0, and InceptionV3. ResNet50 and InceptionV3 have significantly more parameters than EfficientNetB0, resulting in varied performance during training and validation. InceptionV3 achieved the highest training and validation accuracy, indicating better generalization, while EfficientNetB0 struggled the most, exhibiting the highest validation loss and the lowest accuracy. ResNet50 performed moderately, with accuracy between that of EfficientNetB0 and InceptionV3.

The confusion matrix analysis revealed that both ResNet50 and EfficientNetB0 had difficulties in correctly predicting 'Normal' cases, showing a high number of false positives for pneumonia. EfficientNetB0 failed to predict any 'Normal' cases correctly, and ResNet50 had very few correct predictions. In contrast, InceptionV3 demonstrated a better balance, correctly predicting a significant number of both 'Normal' and 'Pneumonia' cases, though it still had notable false negatives and false positives.

ResNet50 and EfficientNetB0 might be overfitting on 'Pneumonia' cases due to the imbalance in predicting 'Normal' cases accurately. InceptionV3, with its deeper and more complex architecture, captures more nuanced features, leading to superior performance. However, there is still room for improvement, especially in reducing false negatives for 'Pneumonia'.

In conclusion, InceptionV3 is the most promising model among the three, with the highest validation accuracy and a better confusion matrix profile. ResNet50 is a decent performer but needs improvements in handling 'Normal' cases, whereas EfficientNetB0 underperformed, suggesting it may require further tuning or data augmentation to be suitable for this specific task. Enhancements could include techniques like data augmentation, class rebalancing, fine-tuning pre-trained weights, and exploring different network architectures.

In [ ]:
# Dictionary of model names and their corresponding models
models = {'ResNet50': resnet50_model, 'EfficentNetB0': efficientNetB0_model, 'InceptionV3': inceptionv3_model}

# Iterate through each model in the dictionary
for model_name, model in models.items():
    # Print the summary of the current model
    model.summary()

    # Evaluate the model on the test dataset and print the validation loss and accuracy
    validation_loss, validation_accuracy = model.evaluate(test_ds)
    print(f'\n Validation Loss: {validation_loss}  Validation Accuracy: {validation_accuracy*100:.2f} %\n')

    # Generate and print the confusion matrix for the current model
    print(f'\nHere is a confusion matrix for {model_name}\n')
    generate_confusion_matrix(model, test_ds)
    
    # Print a separator for better readability
    print('\n--------------------------------------------------------\n')